<a href="https://colab.research.google.com/github/julianzr27/Business-Intelligence/blob/main/Reto_2_%C3%81rbol_de_decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reto 2- Árboles de decisión**

**Descripción de los datos**

De acuerdo con la base de datos de admisión universitaria, esta posee un total de 1528 registros de estudiantes, divididos en 1303 admitidos y 225 no admitidos. Cada uno de los solicitantes de admisión está descrito en función de 7 variables socioeconómicas:

- Edu_Parent1: Nivel educativo alcanzado por el primer padre o tutor.
- Edu_Parent2: Nivel educativo alcanzado por el segundo padre o tutor.
- Gender: Género del estudiante, representado numéricamente como 0 = Femenino, 1 = Masculino.
- White: Variable categórica que indica si el estudiante se identifica como blanco (1 = Sí, 0 = No).
- Asian: Variable categórica que indica si el estudiante se identifica como asiático (1 = Sí, 0 = No).
- HSGPA: Promedio de calificaciones del estudiante en la escuela secundaria.
- SAT/ACT: Puntaje obtenido en los exámenes estandarizados SAT o ACT.
- Admitted: Indica si el estudiante fue admitido en la universidad (1 = Sí, 0 = No).

El análisis se centrará en estas variables para predecir la probabilidad de admisión mediante un modelo basado en árboles de decisión.

0. Se proceden a cargar las librerías

In [ ]:
import numpy as np
import pandas as pd

#Librerías del árbol
from  sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de datos

In [ ]:
nxl ="/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx"
XDB = pd.read_excel(nxl, sheet_name=0)
XDB=XDB.dropna()
XDB.head(100)

,Applicant,Edu_Parent1,Edu_Parent2,Gender,White,Asian,HSGPA,SAT/ACT,College,Admitted,Enrolled,College_GPA
6,7,6,6,F,1,0,4.26,1430,Math & Science,Yes,Yes,3.956
28,29,7,6,M,1,0,3.95,1340,Math & Science,Yes,Yes,3.361
46,47,3,6,M,1,0,4.33,1200,Business & Economics,No,Yes,3.433
61,62,7,7,F,1,0,4.27,1530,Business & Economics,No,Yes,3.890
65,66,6,7,F,0,0,4.04,1270,Math & Science,Yes,Yes,3.025
...,...,...,...,...,...,...,...,...,...,...,...,...
552,553,7,5,F,1,0,4.17,1190,Arts & Letters,No,Yes,3.788
555,556,6,4,F,0,0,3.62,1180,Math & Science,Yes,Yes,3.230
558,559,4,7,F,1,0,3.91,1190,Arts & Letters,Yes,Yes,2.686
564,565,6,4,F,1,0,4.35,1210,Math & Science,Yes,Yes,3.830


1.1. Se seleccionan las variables de interés

In [ ]:
# Convertir la variable Gender a numérica (F:0, M:1)
XDB["Gender"] = XDB["Gender"].map({"F": 0, "M": 1})

# Seleccionar las variables
XD = XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd = XDB[['Admitted']]


1.2. Se determinan el número de personas que fueron admitidas a la Universidad

In [ ]:
# Convertir 'Yes' a 1 y 'No' a 0
XDB["Admitted"] = XDB["Admitted"].map({"Yes": 1, "No": 0})

# Ahora sí, contar los admitidos
num_admitidos = XDB["Admitted"].sum()
print(f"Número de personas admitidas: {num_admitidos}")


Número de personas admitidas: 1303


2. Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4)
mar.fit(XD,yd)

#Se procede con la gráfica
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

ve=['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']
dot_data=export_graphviz(mar, feature_names=ve)
graph=graph_from_dot_data(dot_data)
graph.write_png('ArbolDecision.png')

True

2.1. Se determina el número de nodos puros para el árbol asociado con la estructura del modelo.

In [ ]:
num_nodos_puros = sum(mar.tree_.impurity == 0)
print(f"Número de nodos puros en el árbol (Gini = 0.0): {num_nodos_puros}")

Número de nodos puros en el árbol (Gini = 0.0): 7


3. Se procede con el pronóstico de los solicitantes que no fueron admitidos




In [ ]:
# Cargar la hoja de pronóstico
XDB0 = pd.read_excel(nxl, sheet_name=1)

# Convertir Gender a numérico
XDB0["Gender"] = XDB0["Gender"].map({"F": 0, "M": 1})

# Crear la base de datos para pronóstico
XD0 = XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd0 = XDB0[['Admitted']]

# Mantener todos los solicitantes
filas = XDB0.index

# Hacer la predicción
y_pred = mar.predict(XD0)

# Crear el informe en Excel
df = pd.DataFrame(XD0)
df["Admitted_Predicted"] = y_pred
df.insert(0, "Solicitante", filas)  # Agregar índice de solicitantes

# Guardar en Excel
df.to_excel("Pronostico_Solicitantes_No_Admitidos.xlsx", index=False)

4. Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #pronóstico base de datos original
cm=confusion_matrix(yd,ydp)
print("La matriz de confusión es: \n",cm)

VN= cm[0,0];FP= cm[0,1];VT= cm[1,0];VP= cm[1,1]

#Métricas de desempeño
Ex= (VP+VN)/(VP+VN+FP+VT) #Exactitud: comportamiento general preaprobando
print("La exactitud del modelo es:\n ",Ex)

TE= (FP+VT)/(VP+VN+FP+VT) #Tasa de error: comportamiento general no preaprobando
print("La tasa de error del modelo es:\n ",TE)

Se= VP/(VP+VT) #Sensibilidad: Comportamiento correcto preaprobando
print("La sensibilidad del modelo es:\n ",Se)

Sp= VN/(VN+FP) #Especificidad: Comportamiento correcto no preaprobado
print("La especificidad del modelo es:\n ",Sp)

Pr=VP/(VP+FP) #Precisión
print("La precisión del modelo es:\n ",Pr)

Neg= VN/(VN+VT) #Precisión negativa: comportamiento correcto no preaprobado
print("La precisión negativa del modelo es:\n ",Neg)

La matriz de confusión es: 
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
  0.8645287958115183
La tasa de error del modelo es:
  0.1354712041884817
La sensibilidad del modelo es:
  0.9984650805832693
La especificidad del modelo es:
  0.08888888888888889
La precisión del modelo es:
  0.8638778220451527
La precisión negativa del modelo es:
  0.9090909090909091


**Análisis de Resultados**

El modelo de árbol de decisión aplicado a la base de datos de 1528 datos sobre admisión universitaria mostró un desempeño sólido en la clasificación de los solicitantes.

La matriz de confusión refleja un alto número de verdaderos positivos (1301) y verdaderos negativos (20), mientras que los falsos positivos (205) y falsos negativos (2) son significativamente menores. Esto indica que el modelo tiene un buen desempeño en la identificación de estudiantes admitidos, aunque su capacidad para detectar correctamente a los no admitidos es baja.

Por otro lado, la exactitud del modelo es del 86.45%, lo que significa que en aproximadamente 8 de cada 10 casos, la predicción del modelo es correcta. Sin embargo, la tasa de error del 13.54% sugiere que aún hay margen de mejora.

Un aspecto ambién relevante es la sensibilidad, que alcanza un valor de 99.84%, demostrando que el modelo es altamente eficiente para predecir correctamente a los estudiantes admitidos. En contraste, la especificidad es de apenas 8.89%, lo que indica que la capacidad del modelo para detectar correctamente a los estudiantes no admitidos es baja.

En cuanto a la precisión, se obtuvo un 86.39%, lo que confirma que el modelo es confiable al predecir la admisión de un estudiante. Sin embargo, la precisión negativa (90.90%) sugiere que, aunque pocos estudiantes son clasificados erróneamente como admitidos, la baja especificidad puede afectar su utilidad en escenarios donde es crucial identificar correctamente a los no admitidos.

A partir de lo anterior, podemos concluir que el modelo presenta un desempeño destacado en la clasificación de estudiantes admitidos, pero su baja especificidad indica que no distingue con la misma eficacia a los rechazados, lo que sugiere que podría llegar a mejorarse si se ajustan ciertos parámetros y si se consideran otras técnicas de balanceo de datos para así mejorar la detección de los no admitidos sin afectar la sensibilidad del modelo.